# Vectorize İşlemi Nedir?

**Makine öğrenmesinde ham veriyi (metin, görüntü, kategorik değişkenler vb.) modelin anlayabileceği sayısal vektörler haline dönüştürme sürecidir.**

### Neden Vectorize Ederiz?

- Çoğu ML algoritması yalnızca sayısal girdilerle çalışır.
- Vektörler, hem veriyi standartlaştırır hem de lineer cebirsel optimizasyonlara olanak tanır.

### Yöntemler Nelerdir?

- **One-Hot Encoding:** Kategorik bir değişkenin her benzersiz değeri için ayrı bir boyut oluşturur. İlgili boyut 1 diğerleri 0 olur.

- **TF-IDF**: Her kelimenin belge içindeki sıklığını(term frequency) hem de tüm korpustaki yaygınlığını göz önüne alarak ağırlıklı sayılar üretir.

- **Count Vectorizer:** Metindeki her kelimenin belge içinde kaç kez geçtiğini sayar ve bu sayıları vektörün bileşenleri yapar.

## TF-IDF Vectorizer Üzerinden Birkaç Örnek Yapalım...

- **TF:** Bir terimin belge içindeki sıklığı. (Term Frequency)

- **IDF:** Bir terimin tüm belgeler arasında ne kadar nadir olduğunu ölçer.

- **TF-IDF Skoru:** TF ile IDF'in çarpımıdır.

#### Örneklendirelim;

| Belge ID | Metin                               |
| :------: | :---------------------------------- |
|   Doc 1  | "makine öğrenmesi harika harika"    |
|   Doc 2  | "vectorize işlemi önemlidir makine" |
|   Doc 3  | "makine vectorize öğrenmesi yöntem" |

Üç adet kısa cümlelerimiz var. Yukarıda bu cümlelerimizi görebiliriz.

1. **Terim Listesi oluşturalım:**

- [ 'makine', 'öğrenmesi', 'harika', 'vectorize', 'işlemi', 'önemlidir', 'yöntem' ]

2. **TF Hesabı yapalım: (DOC 1 için)**

- Toplam kelime sayısı: 3

- TF (makine, Doc 1): 1 / 3

- TF (harika, Doc 1): 2 / 3

3. **IDF hesapları:**

- N = 3

- IDF(makine) = log(3 / ( 1 + 3)) ≈ -0.287

- IDF(öğrenmesi) = log(3 / (1+2)) = log(3/3) = 0

- IDF(harika) = log(3 / (1+1)) = log(3/2) ≈ 0.405

4. **TF x IDF = TF-IDF**

- TF-IDF(harika, Doc1) ≈ 0.67 × 0.405 ≈ 0.27

- TF-IDF(makine, Doc1) ≈ 0.33 × (–0.287) ≈ –0.095

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spam_ham_dataset.csv")
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,3624,ham,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\nthis deal is to ...,0


In [3]:
df = df[["text", "label_num"]]
df.head()

,text,label_num
0,Subject: enron methanol ; meter # : 988291\nth...,0
1,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\nthis deal is to ...,0


In [4]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mehmetsolak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import string
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()

def clean_text(text:str):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    words = text.split()
    words = [word for word in words if not word.isdigit()]
    words = [ps.stem(word) for word in words if word not in stopwords.words("english")]
    return " ".join(words)

In [7]:
df["clean_text"] = df["text"].apply(clean_text)
df.head()

,text,label_num,clean_text
0,Subject: enron methanol ; meter # : 988291\nth...,0,subject enron methanol meter follow note gave ...
1,"Subject: hpl nom for january 9 , 2001\n( see a...",0,subject hpl nom januari see attach file hplnol...
2,"Subject: neon retreat\nho ho ho , we ' re arou...",0,subject neon retreat ho ho ho around wonder ti...
3,"Subject: photoshop , windows , office . cheap ...",1,subject photoshop window offic cheap main tren...
4,Subject: re : indian springs\nthis deal is to ...,0,subject indian spring deal book teco pvr reven...


In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

X = df["clean_text"]
y = df["label_num"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf = TfidfVectorizer()
model = MultinomialNB()

pipeline = Pipeline([
    ("tfidf", tfidf),
    ("model", model)
])

param_grid = {
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__min_df': [1, 3, 5, 7],
    'tfidf__max_df': [0.7, 0.8, 0.9, 1.0],
    'tfidf__max_features': [None, 5000, 10000],
}

grid = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

grid.fit(X_train, y_train)
print("Best parameters found: ", grid.best_params_)
print("Best cross-validation score: ", grid.best_score_)
print("Test set score: ", grid.score(X_test, y_test))

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END tfidf__max_df=0.7, tfidf__max_features=None, tfidf__min_df=1, tfidf__ngram_range=(1, 1); total time=   0.2s
[CV] END tfidf__max_df=0.7, tfidf__max_features=None, tfidf__min_df=1, tfidf__ngram_range=(1, 1); total time=   0.2s
[CV] END tfidf__max_df=0.7, tfidf__max_features=None, tfidf__min_df=1, tfidf__ngram_range=(1, 1); total time=   0.2s
[CV] END tfidf__max_df=0.7, tfidf__max_features=None, tfidf__min_df=1, tfidf__ngram_range=(1, 1); total time=   0.2s
[CV] END tfidf__max_df=0.7, tfidf__max_features=None, tfidf__min_df=1, tfidf__ngram_range=(1, 1); total time=   0.2s
[CV] END tfidf__max_df=0.7, tfidf__max_features=None, tfidf__min_df=3, tfidf__ngram_range=(1, 1); total time=   0.2s
[CV] END tfidf__max_df=0.7, tfidf__max_features=None, tfidf__min_df=3, tfidf__ngram_range=(1, 1); total time=   0.2s
[CV] END tfidf__max_df=0.7, tfidf__max_features=None, tfidf__min_df=3, tfidf__ngram_range=(1, 1); total time=   0.2s
[C

### Performans İyileştirmesi İçin Neler Yaptık?

- Pipeline, GridSearch kullanarak en iyi parametreleri buluyoruz. TF-IDF tarafında en iyi parametreleri bulmak için GridSearch içerisine birkaç tane parametre ekledim. Bu sayede tüm parametre kombinasyonları deneyip en yüksek doğruluk veren parametreleri elde ettim.

- **ngram_range=(1, 2) ve (1,3):** Tekil kelimeler (unigram) bazen bağlamı yakalayamaz; bigram veya trigram’lar (“machine learning”, “spam email classification”) modellerin iki-kaç kelime arasındaki anlam ilişkisini öğrenmesini sağlar. Kelime çiftlerini ve üçlüleri de yakalayarak modelin bağlam bilgisini arttırdım. 

- **max_features=5000:** Çok yüksek boyutlu veri aşırı öğrenmeye ve yavaşlığa yol açar. max_features burada en sık geçen veya en yüksek TF-IDF değerine sahip ilk K(5000 mesela) terimi seçer.

- **min_df / max_df:** min_df değeri en az kaç belgede geçmesi gerektiğini ve max_df ise en fazla kaç belgede geçmesi gerektiğini belirten parametrelerdir. Yüzdelik olarak düşünebiliriz, min_df=0.6 dersek minimum olarak tüm belgelerin %60'ında geçmeli demiş oluruz.

**Bu sayede yaklaşık olarak doğruluk oranını 0.92'den 0.97'e çıkarmış olduk.**